In [26]:
%matplotlib inline

import os
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob

from random import randint, shuffle
from skimage.feature import hog

##Data Explorer

###- Feature Extraction
#### * object-detection-crowdai dataset
#### * object-dataset

In [2]:
data_crowdai = pd.read_csv('object-detection-crowdai/labels.csv', header=0)
data_crowdai['Path'] = './object-detection-crowdai/'+ data_crowdai['Frame']
data_crowdai = data_crowdai.drop('Preview URL', 1)
data_crowdai = data_crowdai.drop('Frame', 1)
data_crowdai['Label'] = data_crowdai['Label'].str.lower()

data_crowdai.columns = ['xmin', 'ymin', 'xmax', 'ymax', 'Label', 'Path']
data_crowdai.head()

,xmin,ymin,xmax,ymax,Label,Path
0,785,533,905,644,car,./object-detection-crowdai/1479498371963069978...
1,89,551,291,680,car,./object-detection-crowdai/1479498371963069978...
2,268,546,383,650,car,./object-detection-crowdai/1479498371963069978...
3,455,522,548,615,truck,./object-detection-crowdai/1479498371963069978...
4,548,522,625,605,truck,./object-detection-crowdai/1479498371963069978...


In [20]:
print("Number of file : ", len(data_crowdai))
print("Unique lables are", np.unique(data_crowdai['Label']))

print("Number of car : ", (data_crowdai['Label'] == 'car').sum())
print("Number of truck : ", (data_crowdai['Label'] == 'truck').sum())
print("Number of pederstrians : ", (data_crowdai['Label'] == 'pedestrian').sum())

Number of file :  72064
Unique lables are ['car' 'pedestrian' 'truck']
Number of car :  62570
Number of truck :  3819
Number of pederstrians :  5675


In [10]:
name = ['Frame', 'xmin', 'ymin', 'xmax', 'ymax', 'ind', 'Label', 'RM']

data_object = pd.read_csv('object-dataset/labels.csv', delim_whitespace=True, names=name)
data_object['Path'] = './object-dataset/' + data_object['Frame']
data_object = data_object.drop('RM', 1)
data_object = data_object.drop('Frame', 1)
data_object = data_object.drop('ind', 1)

data_object.columns = ['xmin', 'ymin', 'xmax', 'ymax', 'Label', 'Path']
data_object.head()

,xmin,ymin,xmax,ymax,Label,Path
0,950,574,1004,620,car,./object-dataset/1478019952686311006.jpg
1,1748,482,1818,744,pedestrian,./object-dataset/1478019952686311006.jpg
2,872,586,926,632,car,./object-dataset/1478019953180167674.jpg
3,686,566,728,618,truck,./object-dataset/1478019953689774621.jpg
4,716,578,764,622,car,./object-dataset/1478019953689774621.jpg


In [21]:
print("Number of file : ", len(data_object))
print("Unique lables are", np.unique(data_object['Label']))

print("Number of car : ", (data_object['Label'] == 'car').sum())
print("Number of truck : ", (data_object['Label'] == 'truck').sum())
print("Number of pederstrians : ", (data_object['Label'] == 'pedestrian').sum())

Number of file :  93086
Unique lables are ['biker' 'car' 'pedestrian' 'trafficLight' 'truck']
Number of car :  60788
Number of truck :  3503
Number of pederstrians :  9866


In [22]:
# Combine each dataset

data_train = pd.concat([data_object, data_object]).reset_index()
data_train = data_train.drop('index', 1)
data_train.columns = ['xmin', 'ymin', 'xmax', 'ymax', 'Label', 'Path']
data_train.head()

,xmin,ymin,xmax,ymax,Label,Path
0,950,574,1004,620,car,./object-dataset/1478019952686311006.jpg
1,1748,482,1818,744,pedestrian,./object-dataset/1478019952686311006.jpg
2,872,586,926,632,car,./object-dataset/1478019953180167674.jpg
3,686,566,728,618,truck,./object-dataset/1478019953689774621.jpg
4,716,578,764,622,car,./object-dataset/1478019953689774621.jpg


In [24]:
cars_s = data_train[(data_train['Label'] == 'car') | (data_train['Label'] == 'truck')].reset_index()
nocars_s = data_train[(data_train['Label'] != 'car') & (data_train['Label'] != 'truck')].reset_index()

print("Number of examples =", len(data_train)) 
print("Number of car examples =", len(cars)) 
print("Number of other examples =", len(nocars))

Number of examples = 186172
Number of car examples = 128582
Number of other examples = 57590


In [27]:
# Sampling dataset

cars = []
nocars = []

sample_size = 5000

# sampling car data in dataset
for i in range(sample_size):
    index = randint(0, len(cars_s)-1)
    image = cv2.imread(cars_s['Path'][index])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image[cars_s['ymin'][index]:cars_s['ymax'][index], 
                  cars_s['xmin'][index]:cars_s['xmax'][index]]
    image = cv2.resize(image, (64, 64), interpolation=cv2.INTER_AREA)
    cars.append(image)

# sampling not car data in dataset
for i in range(sample_size):
    index = randint(0, len(nocars_s)-1)
    image = cv2.imread(nocars_s['Path'][index])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image[nocars_s['ymin'][index]:nocars_s['ymax'][index], 
                  nocars_s['xmin'][index]:nocars_s['xmax'][index]]
    image = cv2.resize(image, (64, 64), interpolation=cv2.INTER_AREA)
    nocars.append(image)